In [1]:
import pandas as pd 
import numpy as np 
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_book_ratings = pd.read_csv("BX-Book-Ratings.csv", delimiter=';', error_bad_lines=False, encoding="latin-1")
data_books = pd.read_csv("BX-Books.csv", delimiter=';', error_bad_lines=False, encoding="latin-1")
data_users = pd.read_csv("BX-Users.csv", delimiter=';', error_bad_lines=False, encoding="latin-1")

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


In [4]:
data_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [5]:
data_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [6]:
data_book_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [7]:
combined_book_rating = data_books.merge(data_book_ratings, left_on="ISBN", right_on="ISBN", how="right")

In [8]:
combined_book_rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1149780 entries, 0 to 1149779
Data columns (total 10 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   ISBN                 1149780 non-null  object
 1   Book-Title           1031136 non-null  object
 2   Book-Author          1031135 non-null  object
 3   Year-Of-Publication  1031136 non-null  object
 4   Publisher            1031134 non-null  object
 5   Image-URL-S          1031136 non-null  object
 6   Image-URL-M          1031136 non-null  object
 7   Image-URL-L          1031132 non-null  object
 8   User-ID              1149780 non-null  int64 
 9   Book-Rating          1149780 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 96.5+ MB


In [9]:
total_book_rating = data_book_ratings.groupby(["ISBN"], as_index=False).sum()

In [10]:
total_book_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340556 entries, 0 to 340555
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ISBN         340556 non-null  object
 1   User-ID      340556 non-null  int64 
 2   Book-Rating  340556 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 7.8+ MB


In [11]:
total_book_rating

,ISBN,User-ID,Book-Rating
0,0330299891,271009,6
1,0375404120,356057,3
2,0586045007,89192,0
3,9022906116,100868,7
4,9032803328,89192,0
...,...,...,...
340551,cn113107,107301,0
340552,ooo7156103,266146,7
340553,§423350229,85250,0
340554,´3499128624,86103,8


In [12]:
total_book_rating = total_book_rating.loc[total_book_rating["Book-Rating"] >= 50]

In [13]:
total_book_rating.shape

(9300, 3)

In [14]:
data_users["Location"] = data_users["Location"].apply(lambda loc: loc.split(',')[-1].strip())

In [15]:
data_users

,User-ID,Location,Age
0,1,usa,NaN
1,2,usa,18.0
2,3,russia,NaN
3,4,portugal,17.0
4,5,united kingdom,NaN
...,...,...,...
278853,278854,usa,NaN
278854,278855,united kingdom,50.0
278855,278856,canada,NaN
278856,278857,usa,NaN


In [16]:
filtered_data_users = data_users[data_users["Location"].isin(["usa", "canada"])]

In [17]:
print(combined_book_rating.shape, total_book_rating.shape)

(1149780, 10) (9300, 3)


In [18]:
combined_book_rating = combined_book_rating[combined_book_rating["ISBN"].isin((total_book_rating["ISBN"]))]
print(combined_book_rating.shape)
combined_book_rating = combined_book_rating[combined_book_rating["User-ID"].isin(filtered_data_users["User-ID"])]
print(combined_book_rating.shape)

(409485, 10)
(344387, 10)


In [19]:
combined_book_rating = combined_book_rating.dropna(subset=["Book-Title"])

In [20]:
combined_book_rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 340560 entries, 0 to 1149777
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 340560 non-null  object
 1   Book-Title           340560 non-null  object
 2   Book-Author          340560 non-null  object
 3   Year-Of-Publication  340560 non-null  object
 4   Publisher            340560 non-null  object
 5   Image-URL-S          340560 non-null  object
 6   Image-URL-M          340560 non-null  object
 7   Image-URL-L          340560 non-null  object
 8   User-ID              340560 non-null  int64 
 9   Book-Rating          340560 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 28.6+ MB


In [21]:
combined_book_rating = combined_book_rating.groupby(by=["Book-Title", "User-ID"], as_index=False).mean()

In [22]:
combined_book_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338880 entries, 0 to 338879
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Book-Title   338880 non-null  object 
 1   User-ID      338880 non-null  int64  
 2   Book-Rating  338880 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 7.8+ MB


In [23]:
combined_book_rating

,Book-Title,User-ID,Book-Rating
0,Earth Prayers From around the World: 365 Pray...,26544,9.0
1,Earth Prayers From around the World: 365 Pray...,69120,10.0
2,Earth Prayers From around the World: 365 Pray...,121592,7.0
3,Earth Prayers From around the World: 365 Pray...,179730,1.0
4,Earth Prayers From around the World: 365 Pray...,179744,6.0
...,...,...,...
338875,stardust,235842,0.0
338876,stardust,236754,10.0
338877,stardust,252724,0.0
338878,stardust,274393,8.0


In [24]:
rating_matrix = combined_book_rating.pivot(index="Book-Title", columns="User-ID", values="Book-Rating")

In [25]:
rating_matrix = rating_matrix.fillna(0)

In [26]:
rating_matrix

User-ID,8,9,14,16,17,23,26,32,39,42,...,278831,278832,278836,278838,278843,278844,278846,278849,278851,278854
Book-Title,,,,,,,,,,,,,,,,,,,,,
"Earth Prayers From around the World: 365 Prayers, Poems, and Invocations for Honoring the Earth",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
!Yo!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01-01-00: The Novel of the Millennium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Lb. Penalty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"\The Happy Prince\"" and Other Stories (Penguin Popular Classics)""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"\What Do You Care What Other People Think?\"": Further Adventures of a Curious Character""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
sparse_rating_matrix = csr_matrix(rating_matrix)

In [28]:
sparse_rating_matrix

<7575x46908 sparse matrix of type '<class 'numpy.float64'>'
	with 137887 stored elements in Compressed Sparse Row format>

In [29]:
findnn = NearestNeighbors(n_neighbors=5)
findnn = findnn.fit(sparse_rating_matrix)

In [30]:
best5 = findnn.kneighbors([rating_matrix.loc["Adventures of Huckleberry Finn (Dover Thrift Editions)", :]], return_distance=False)
for sug in best5:
    for ind in sug:
        print(rating_matrix.index[ind-1])

Adventures of Huckleberry Finn
Le Parfum : Histoire d'un meurtrier
Der Kleine Hobbit
Zen in the Art of Writing
Strapless (Red Dress Ink (Paperback))
